In [10]:
import pandas as pd 
from pandas.io.json import json_normalize
from ast import literal_eval

In [11]:
MAXROWS = 1e5 # per chunk

In [9]:
df_list = []
for file in ['data/train_v2.csv', 'data/test_v2.csv']:
    reader = pd.read_csv(file, usecols=['fullVisitorId','visitStartTime', 'hits'], chunksize = MAXROWS, skiprows=0, low_memory=False)
    for chunk in reader:
        chunk.columns = ['hits']
        chunk['hits'][chunk['hits'] == "[]"] = "[{}]"
        chunk['hits'] = chunk['hits'].apply(literal_eval).str[0]
        chunk = json_normalize(chunk['hits'])

        # Extract the product and promo names from the complex nested structure into a simple flat list:
        if 'product' in chunk.columns:
            #print(chunk['product'][0])
            chunk['v2ProductName'] = chunk['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
            chunk['v2ProductCategory'] = chunk['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
            del chunk['product']
        if 'promotion' in chunk.columns:
            #print(chunk['promotion'][0])
            chunk['promoId']  = chunk['promotion'].apply(lambda x: [p['promoId'] for p in x] if type(x) == list else [])
            chunk['promoName']  = chunk['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
            del chunk['promotion']

        df_list.append(chunk)
        print(f"Chunk {len(df_list)} completed: {len(chunk)} rows from '{file}'")

ValueError: Length mismatch: Expected axis has 1 elements, new values have 3 elements

In [10]:
df = pd.concat(df_list, ignore_index = True, copy = False, join = "outer", sort = True)

In [12]:
df.shape

(2109926, 79)

In [2]:
df.head

NameError: name 'df' is not defined

In [ ]:
df.to_csv('data/concat_hit.csv')